# Cargando lo que ya teníamos

In [1]:
import pandas as pd
import csv

In [9]:
df = pd.read_csv('corpus.csv')

## Distribución de palabras

Para entender un poco más el problema veremos la distribución de las palabras. Además esto nos va a ayudar a evitar problemas en un futuro.

In [3]:
from collections import Counter
from itertools import chain

In [10]:
distribution_words = Counter(chain.from_iterable(df.message.map(lambda x: x.split(' '))))

In [11]:
df_distribution = pd.DataFrame(data={'word': list(distribution_words.keys()), 'counter': list(distribution_words.values())})

Es decir el vocabulario total es alrededor de:

In [12]:
df_distribution.count()

counter    101760
word       101760
dtype: int64

Por lo que las palabras menos vistas son

In [13]:
df_distribution.sort_values('counter').head(20)

,counter,word
50879,1,fallabela
65636,1,boloooooooo
65635,1,Humble
65633,1,sin.hacer
65632,1,dinero.en
65631,1,muxo
65630,1,xno
65629,1,siendo.actor
65628,1,Jajaajjaaajajaja
65627,1,nickelodeon


Pero las más vistas son:

In [14]:
df_distribution.sort_values('counter', ascending=False).head(20)

,counter,word
12,24707,que
37,24333,de
16,19957,a
23,18938,la
46,17263,el
96,14856,no
31,13411,y
29,12576,en
2805,11407,<Archivo
2806,11407,omitido>


Al querer hacer un gráfico de caja, nos dimos cuenta que la media en la distribución de palabras era 1...

In [13]:
df_distribution.counter.median()

1.0

Pues luego de ver esa que la media era 1, nos preguntamos...
   - ¿Cuántas palabras hay con una sóla aparición? Son útiles o no?

In [14]:
sum(df_distribution.counter == 1) / df_distribution.count()

counter    0.655504
word       0.655504
dtype: float64

Vemos que hay alrededor de un 65% de palabras que aparecen una sóla vez... esto puede causar que la **vectorización no sea tan representativa como queremos...**

Problemas que vimos:
   - Al ser mensajes instantaneos, por lo general las personas no se preocupan con la ortográfia ni la forma de escribir.
   - Muchos tipos de mismas palabras, por ejemplo jajaja = jaja, dale = daleee.
   - _Arhivo omitido_ como media no es útil.
   - Puntaciones son inútiles para el problema (no ayudan en nada). Ejemplo quiero = quiero'
   - Los emoticones son necesarios para este problema?

## Conclusión

Después de discutir un rato decidimos tomar estás decisiones para mejorar la distribución:
   - "Normalizar palabras más vistas" (jajajajajaa --> jaja)
   - Tratar las puntuaciones como palabras ("?", "!", "...") o directamente removerlas (quiero' --> quiero)
   - Pasar a minuscula toda palabra
   - Eliminar palabra que no aparezca más de 10 veces (soluciona errores de ortografía, links, etc)
   - Remover acentos
   - Pasar digito a la palabra NUM
   - Remover characteres repetidos, ejemplo: vaaaaamos por vamos.
   - __Hacer Stemm serviría? FIJARSE __

Veamos como sería la nueva distribución

In [20]:
from string import punctuation
import re
import unicodedata

In [44]:
STOP_WORDS = punctuation

def dig2num(text):
    return re.sub('(\d+[\WxX]*[^ ]*)+', 'NUM', text)

def rm_tildes(text):
    return ''.join((c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn'))

def rm_media(text):
    return re.sub(r'<archivo omitido>|<media omitted>', 'MEDIA', text)

def rm_repeated(text):
    return re.sub(r'([aeioursh])\1{2,}', r'\1', text)

def rm_ok(text):
    return re.sub('(ok)+[akis]*', 'ok', text)

def rm_sw(text):
    return re.sub('{1}{0}{1}'.format(STOP_WORDS, '[]'), ' ', text)

def rm_ja(text):
    return re.sub('(ja[ja(js)]*)+', 'jaja', text)

def clean_text(text):
    return dig2num(rm_repeated(rm_ok(rm_ja(rm_tildes(rm_sw(rm_media(text.lower())))))))

In [45]:
distribution_words = Counter(chain.from_iterable(df.message.map(lambda x: clean_text(x).split())))

In [46]:
df_distribution = pd.DataFrame(data={'word': list(distribution_words.keys()), 'counter': list(distribution_words.values())})

In [47]:
new_df = df_distribution[df_distribution.counter > 10]

In [48]:
new_df.count()

counter    5648
word       5648
dtype: int64

In [49]:
new_df.sort_values('counter', ascending=False).head(20)

,counter,word
9,29623,que
34,25788,de
13,22575,a
86,22451,no
20,21036,la
187,19937,jaja
42,19887,el
28,18444,y
71,17831,NUM
0,16066,MEDIA


In [50]:
new_df.sort_values('counter').head(20)

,counter,word
26876,11,yankee
13675,11,chi
13695,11,sisis
9216,11,parecidos
13708,11,raros
23344,11,yep
23410,11,cupo
13743,11,descanso
9144,11,adelantar
19730,11,gol


Las listas van cobrando sentido